<a href="https://colab.research.google.com/github/shiladeori0305/prediction_model/blob/main/prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok yfinance scikit-learn matplotlib --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.3 MB/s eta 0:00:00


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np


In [3]:
tickers = ["GLD", "SLV", "USO", "SPY", "DX-Y.NYB"]

data = yf.download(tickers, start="2012-01-01", end="2024-12-31")['Close']
data.head()


/tmp/ipython-input-549737067.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2012-01-01", end="2024-12-31")['Close']
[*********************100%***********************]  5 of 5 completed


Ticker,DX-Y.NYB,GLD,SLV,SPY,USO
Date,,,,,
2012-01-03,79.610001,155.919998,28.830000,99.583405,317.519989
2012-01-04,80.089996,156.710007,28.389999,99.739647,318.160004
2012-01-05,80.940002,157.779999,28.510000,100.005203,313.359985
2012-01-06,81.239998,157.199997,27.910000,99.747459,313.760010
2012-01-09,81.050003,156.500000,28.129999,99.989571,312.640015


In [4]:
future_days = 10

df = data.copy()
for col in df.columns:
    df[f"{col}_pred"] = df[col].shift(-future_days)

df.dropna(inplace=True)
df.head()


Ticker,DX-Y.NYB,GLD,SLV,SPY,USO,DX-Y.NYB_pred,GLD_pred,SLV_pred,SPY_pred,USO_pred
Date,,,,,,,,,,
2012-01-03,79.610001,155.919998,28.830000,99.583405,317.519989,80.599998,161.600006,29.650000,102.137459,310.640015
2012-01-04,80.089996,156.710007,28.389999,99.739647,318.160004,80.209999,161.220001,29.740000,102.676384,309.519989
2012-01-05,80.940002,157.779999,28.510000,100.005203,313.359985,80.220001,162.070007,31.219999,103.059082,302.239990
2012-01-06,81.239998,157.199997,27.910000,99.747459,313.760010,79.690002,163.160004,31.389999,102.793549,307.119995
2012-01-09,81.050003,156.500000,28.129999,99.989571,312.640015,79.860001,162.009995,31.080000,102.676384,304.799988


In [5]:
from sklearn.ensemble import RandomForestRegressor

models = {}

for col in data.columns:
    X = df[[col]]
    y = df[f"{col}_pred"]

    model = RandomForestRegressor(n_estimators=200)
    model.fit(X, y)

    models[col] = model

print("Models trained for:", models.keys())


Models trained for: dict_keys(['DX-Y.NYB', 'GLD', 'SLV', 'SPY', 'USO'])


In [6]:
!pip install plotly --quiet


In [7]:
!pip install streamlit pyngrok yfinance scikit-learn plotly beautifulsoup4 requests


In [8]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestRegressor
import plotly.graph_objects as go
from datetime import timedelta

st.set_page_config(layout="wide")
st.title("Indian Commodities Live Dashboard & Prediction")

# ----------------------------- #
# -------- SCRAPERS ---------- #
# ----------------------------- #

def scrape_gold_24k_10g_delhi():
    try:
        url = "https://www.goodreturns.in/gold-rates/delhi.html"
        soup = BeautifulSoup(requests.get(url, timeout=10).text, "html.parser")
        table = soup.find("div", {"class": "gold_silver_table"})
        for line in table.text.split("\n"):
            if "24 Carat" in line and "10g" in line:
                return float(line.split("₹")[1].replace(",", "").strip())
    except:
        return None


def scrape_silver_1kg_delhi():
    try:
        url = "https://www.goodreturns.in/silver-rates/delhi.html"
        soup = BeautifulSoup(requests.get(url, timeout=10).text, "html.parser")
        table = soup.find("div", {"class": "gold_silver_table"})
        for line in table.text.split("\n"):
            if "1 Kg" in line and "₹" in line:
                return float(line.split("₹")[1].replace(",", "").strip())
    except:
        return None


# ----------------------------- #
# ------- MARKET DATA -------- #
# ----------------------------- #

@st.cache_data(ttl=3600)
def load_data():
    tickers = {
        "GC=F": "Gold_USD",
        "SI=F": "Silver_USD",
        "CL=F": "Crude",
        "NG=F": "NatGas",
        "HG=F": "Copper",
        "USDINR=X": "USDINR",
    }

    df = pd.DataFrame()
    for t, name in tickers.items():
        df[name] = yf.download(t, period="1y", interval="1d")["Close"]

    df.dropna(inplace=True)
    return df


data = load_data()

# ----------------------------- #
# ----- USD → INR MODEL ------ #
# ----------------------------- #

def gold_inr_10g(usd_oz, usd_inr):
    return (usd_oz * usd_inr * 10) / 31.1035


def silver_inr_kg(usd_oz, usd_inr):
    return (usd_oz * usd_inr * 1000) / 31.1035


data["Gold_Model"] = gold_inr_10g(data["Gold_USD"], data["USDINR"])
data["Silver_Model"] = silver_inr_kg(data["Silver_USD"], data["USDINR"])
data["Crude_Model"] = data["Crude"] * data["USDINR"]
data["NatGas_Model"] = data["NatGas"] * data["USDINR"]
data["Copper_Model"] = data["Copper"] * data["USDINR"]

# ----------------------------- #
# ----- FEATURE ENGINEERING -- #
# ----------------------------- #

def make_features(df, col):
    d = df.copy()

    for i in range(1, 6):
        d[f"lag_{i}"] = d[col].shift(i)

    d["roll_mean"] = d[col].rolling(3).mean()
    d["roll_std"] = d[col].rolling(3).std()
    d["USDINR_feat"] = d["USDINR"]
    d["Crude_feat"] = d["Crude_Model"]

    d["Target"] = d[col].shift(-1)
    d.dropna(inplace=True)

    return d


# ----------------------------- #
# -------- ML MODEL ---------- #
# ----------------------------- #

def train_predict(df, col):
    feat = make_features(df, col)

    features = [
        col, "lag_1", "lag_2", "lag_3", "lag_4", "lag_5",
        "roll_mean", "roll_std",
        "USDINR_feat", "Crude_feat"
    ]

    X = feat[features]
    y = feat["Target"]

    model = RandomForestRegressor(
        n_estimators=400,
        max_depth=12,
        random_state=42
    )
    model.fit(X, y)

    last = feat.iloc[-1][features].values.reshape(1, -1)

    preds, dates = [], []
    base = df.index[-1]

    for i in range(5):
        p = model.predict(last)[0]
        preds.append(p)
        dates.append(base + timedelta(days=i + 1))

        last = np.roll(last, 1)
        last[0][0] = p

    return preds, dates


# ----------------------------- #
# ----------- UI ------------- #
# ----------------------------- #

commodity_map = {
    "Gold (24K, ₹/10g)": "Gold_Model",
    "Silver (₹/kg)": "Silver_Model",
    "Crude Oil": "Crude_Model",
    "Natural Gas": "NatGas_Model",
    "Copper": "Copper_Model",
}

choice = st.selectbox("Select Commodity", list(commodity_map.keys()))
col = commodity_map[choice]

# Live Price
if "Gold" in choice:
    live = scrape_gold_24k_10g_delhi()
    price = live if live else data[col].iloc[-1]
    st.metric("Delhi Gold 24K (10g)", f"₹ {price:,.2f}")

elif "Silver" in choice:
    live = scrape_silver_1kg_delhi()
    price = live if live else data[col].iloc[-1]
    st.metric("Delhi Silver (1 Kg)", f"₹ {price:,.2f}")

else:
    price = data[col].iloc[-1]
    st.metric("Current Price (INR)", f"₹ {price:,.2f}")

# Prediction
preds, dates = train_predict(data, col)

# Graph
last10 = data.tail(10)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=last10.index, y=last10[col],
    mode="lines+markers", name="Last 10 Days"
))
fig.add_trace(go.Scatter(
    x=dates, y=preds,
    mode="lines+markers", name="Next 5 Days Prediction"
))

fig.update_layout(
    title=choice,
    xaxis_title="Date",
    yaxis_title="Price (INR)"
)

st.plotly_chart(fig, width="stretch")

# Table
st.subheader("Next 5 Days Predicted Prices")
st.dataframe(pd.DataFrame({
    "Date": dates,
    "Predicted Price (INR)": preds
}))


Writing app.py


In [9]:
!ngrok config add-authtoken 39WpPhs0tY7IvzaO9tec98G5S3q_3k9RKFqyxCApfhE6XPWnR

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
!pkill -f streamlit
!pkill -f ngrok

get_ipython().system_raw('streamlit run app.py --server.port 8501 &')

from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(public_url)


NgrokTunnel: "https://charitably-crossable-providencia.ngrok-free.dev" -> "http://localhost:8501"
